In [21]:
import re
import subprocess
from itertools import compress
import numpy as np

In [6]:
print("Before =========================")
subprocess.run(["cat", "tests/deletion/del.sam"])
print("=========================")

Before =========================
@SQ	SN:random_100bp	LN:100
@PG	ID:minimap2	PN:minimap2	VN:2.22-r1101	CL:minimap2 -ax map-ont tests/random_100bp.fa tests/deletion/del.fq
control	0	random_100bp	1	60	100M	*	0	0	ACTGTGCGGCATACTTAATTATACATTTGAAACGCGCCCAAGTGACGCTAGGCAAGTCAGAGCAGGTTCCCGTGTTAGCTTAAGGGTAAACATACAAGTC	!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!	NM:i:0	ms:i:200	AS:i:200	nn:i:0	tp:A:P	cm:i:15	s1:i:95	s2:i:0	de:f:0	rl:i:0
del-10nt	0	random_100bp	1	60	39M10D51M	*	0	0	ACTGTGCGGCATACTTAATTATACATTTGAAACGCGCCCAGGCAAGTCAGAGCAGGTTCCCGTGTTAGCTTAAGGGTAAACATACAAGTC	!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!	NM:i:10	ms:i:169	AS:i:156	nn:i:0	tp:A:P	cm:i:11	s1:i:79	s2:i:0	de:f:0.0110	rl:i:0


In [123]:
print("CS=========================")
subprocess.run(["cat", "tests/deletion/del_cs.sam"])
print("=========================")

CS=========================
@SQ	SN:random_100bp	LN:100
@PG	ID:minimap2	PN:minimap2	VN:2.22-r1101	CL:minimap2 -ax map-ont --cs tests/random_100bp.fa tests/deletion/del.fq
control	0	random_100bp	1	60	100M	*	0	0	ACTGTGCGGCATACTTAATTATACATTTGAAACGCGCCCAAGTGACGCTAGGCAAGTCAGAGCAGGTTCCCGTGTTAGCTTAAGGGTAAACATACAAGTC	!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!	NM:i:0	ms:i:200	AS:i:200	nn:i:0	tp:A:P	cm:i:15	s1:i:95	s2:i:0	de:f:0	cs:Z::100	rl:i:0
del-10nt	0	random_100bp	1	60	39M10D51M	*	0	0	ACTGTGCGGCATACTTAATTATACATTTGAAACGCGCCCAGGCAAGTCAGAGCAGGTTCCCGTGTTAGCTTAAGGGTAAACATACAAGTC	!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!	NM:i:10	ms:i:169	AS:i:156	nn:i:0	tp:A:P	cm:i:11	s1:i:79	s2:i:0	de:f:0.0110	cs:Z::39-aagtgacgct:51	rl:i:0


In [124]:
print("CS long=========================")
subprocess.run(["cat", "tests/deletion/del_cslong.sam"])
print("=========================")

CS long=========================
@SQ	SN:random_100bp	LN:100
@PG	ID:minimap2	PN:minimap2	VN:2.22-r1101	CL:minimap2 -ax map-ont --cs=long tests/random_100bp.fa tests/deletion/del.fq
control	0	random_100bp	1	60	100M	*	0	0	ACTGTGCGGCATACTTAATTATACATTTGAAACGCGCCCAAGTGACGCTAGGCAAGTCAGAGCAGGTTCCCGTGTTAGCTTAAGGGTAAACATACAAGTC	!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!	NM:i:0	ms:i:200	AS:i:200	nn:i:0	tp:A:P	cm:i:15	s1:i:95	s2:i:0	de:f:0	cs:Z:=ACTGTGCGGCATACTTAATTATACATTTGAAACGCGCCCAAGTGACGCTAGGCAAGTCAGAGCAGGTTCCCGTGTTAGCTTAAGGGTAAACATACAAGTC	rl:i:0
del-10nt	0	random_100bp	1	60	39M10D51M	*	0	0	ACTGTGCGGCATACTTAATTATACATTTGAAACGCGCCCAGGCAAGTCAGAGCAGGTTCCCGTGTTAGCTTAAGGGTAAACATACAAGTC	!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!	NM:i:10	ms:i:169	AS:i:156	nn:i:0	tp:A:P	cm:i:11	s1:i:79	s2:i:0	de:f:0.0110	cs:Z:=ACTGTGCGGCATACTTAATTATACATTTGAAACGCGCCC-aagtgacgct=AGGCAAGTCAGAGCAGGTTCCCGTGTTAGCTTAAGGG

## parse query sam

In [9]:
que_sam = []
with open("tests/deletion/del.sam") as f:
    for s in f:
        row = s.strip()
        que_sam.append(row)

header_idx = [_.startswith("@") for _ in que_sam]
body_idx = [not _ for _ in header_idx]

header = list(compress(que_sam, header_idx))
body = list(compress(que_sam, body_idx))

cigar = [s.split("\t")[5] for s in body]
que_seq = [s.split("\t")[9] for s in body]

In [10]:
print(cigar)
print(que_seq)

['100M', '39M10D51M']
['ACTGTGCGGCATACTTAATTATACATTTGAAACGCGCCCAAGTGACGCTAGGCAAGTCAGAGCAGGTTCCCGTGTTAGCTTAAGGGTAAACATACAAGTC', 'ACTGTGCGGCATACTTAATTATACATTTGAAACGCGCCCAGGCAAGTCAGAGCAGGTTCCCGTGTTAGCTTAAGGGTAAACATACAAGTC']


## parce reference fasta

In [111]:
ref_fasta = []
with open("tests/random_100bp.fa") as f:
    for s in f:
        row = s.strip()
        ref_fasta.append(row)

ref_seq_idx = [not _.startswith(">") for _ in ref_fasta]
ref_seq = list(compress(ref_fasta, ref_seq_idx))[0]

In [112]:
print(ref_seq)

ACTGTGCGGCATACTTAATTATACATTTGAAACGCGCCCAAGTGACGCTAGGCAAGTCAGAGCAGGTTCCCGTGTTAGCTTAAGGGTAAACATACAAGTC


## Format CIGAR

In [126]:
def cigar_split(s):
    group_by = 2
    s_split = re.split(r"(\d+)", s)[1:]
    s_group_by = [s_split[i:i + group_by]
                  for i in range(0, len(s_split), group_by)]
    return s_group_by


cigar = [cigar_split(s) for s in cigar]

In [127]:
print(cigar)
# print(int(cigar[0][0][0]) * cigar[0][0][1])

# print(int(cigar[1][0][0]) * cigar[1][0][1])
# print(int(cigar[1][1][0]) * cigar[1][1][1])
# print(int(cigar[1][2][0]) * cigar[1][2][1])

[[], []]


In [68]:
# 高速化の余地がありそうです
def extend_cigar(cigar): 
    extend = []
    for each_read in cigar:
        tmp_extend = []
        for num, char in each_read:
            tmp_extend.append(int(num) * char)
        extend.append(''.join(tmp_extend))
    return extend

In [74]:
%time
cigar = extend_cigar(cigar)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.81 µs


In [125]:
print(cigar)

['MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM', 'MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMDDDDDDDDDDMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM']


## Annotate indel in que_seq using CIGAR

In [81]:
print(ref_seq)
print(cigar)

['ACTGTGCGGCATACTTAATTATACATTTGAAACGCGCCCAAGTGACGCTAGGCAAGTCAGAGCAGGTTCCCGTGTTAGCTTAAGGGTAAACATACAAGTC']
['MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM', 'MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMDDDDDDDDDDMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM']


In [115]:
tmp_cigar = cigar[1]


In [122]:
for idx, char in enumerate(tmp_cigar):
    tmp_ref_seq = list(ref_seq)
    if char == "D":
        print(tmp_ref_seq[idx])
        print(tmp_ref_seq[idx].lower())
        tmp_ref_seq[idx] = tmp_ref_seq[idx].lower()
print("".join(tmp_ref_seq))

A
a
A
a
G
g
T
t
G
g
A
a
C
c
G
g
C
c
T
t
ACTGTGCGGCATACTTAATTATACATTTGAAACGCGCCCAAGTGACGCTAGGCAAGTCAGAGCAGGTTCCCGTGTTAGCTTAAGGGTAAACATACAAGTC


In [110]:
s = list(tmp_ref_seq)
s[10] = tmp_ref_seq[10].lower()
print("".join(s))

ACTGTGCGGCaTACTTAATTATACATTTGAAACGCGCCCAAGTGACGCTAGGCAAGTCAGAGCAGGTTCCCGTGTTAGCTTAAGGGTAAACATACAAGTC


## Annotate substitution in que_seq using reference fasta